# Assignment 2

## Linear Regression and coeffcient of corelation

In [2]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

### Importing Data from CSV File
Using PANDAS to import data from iris.csv

In [3]:
data = pd.read_csv("iris.csv")

### Calculating the Pearon's correlation
Finding the pearson's coefficient for the following pairs:
    
**1) Sepal Length and Petal Width**

**2) Petal Length and Petal Width**

We will use the **.corr()** function to find the correlation between the two features. Passing the **method = 'pearson'** argument to find the pearson's correlation coefficient

In [4]:
correlation1 = data["sepal_length"].corr(data["petal_width"], method='pearson')
correlation2 = data["petal_length"].corr(data["petal_width"], method='pearson')

print("Sepal Length and Petal Width:", correlation1)
print("Petal Length and Petal Width:", correlation2)

Sepal Length and Petal Width: 0.8179411262715754
Petal Length and Petal Width: 0.962865431402796


### Choosing the independent variable
We can see that the correlation coefficient between **Petal Length & Petal Width** is greater, so we'll use **Petal Length as the independent vairble in the first model**

In [5]:
Y = data["petal_width"].values  # Dependent variable

# Choosing the indepent variable
if correlation1 > correlation2:
    X1 = data["sepal_length"].values
    X2 = data["petal_length"].values
    print('Sepal Length chosen as the indepent variable for the first model, and Petal Length for the second model')
else:
    X1 = data["petal_length"].values
    X2 = data["sepal_length"].values
    print('Petal Length chosen as the indepent variable for the first model, and Sepal Length for the second model')

Petal Length chosen as the indepent variable for the first model, and Sepal Length for the second model


## Model 1
Making a pymc model, in this case with **petal_width as dependent vairable** & **Petal Length as independent variable**

In [6]:
with pm.Model() as model1:
  α = pm.Normal('α1', mu=0, sigma=10)
  β = pm.Normal('β1', mu=0, sigma=1)
  ε = pm.HalfCauchy('ε1', 5)
  μ = pm.Deterministic('μ1', α + β * X1)  # To include the origonal values in the trace
  y1_pred = pm.Normal('y1_pred', mu=μ, sigma=ε, observed=Y)
  trace1 = pm.sample(1000, chains=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [α1, β1, ε1]


ValueError: Not enough samples to build a trace.

## Model 2
Making a pymc model, in this case with **petal_width as dependent vairable** & **Sepal Length as independent variable**

In [8]:
with pm.Model() as model2:
  α = pm.Normal('α1', mu=0, sigma=10)
  β = pm.Normal('β1', mu=0, sigma=1)
  ε = pm.HalfCauchy('ε1', 5)
  μ = pm.Deterministic('μ1', α + β * X2)  # To include the origonal values in the trace
  y2_pred = pm.Normal('y2_pred', mu=μ, sigma=ε, observed=Y)
  trace2 = pm.sample(1000, chains=2)

### Calculating the R-Squared Values for both models
We can calculate the R-Squared Values using **az.r2_score()** function in the arviz library. But first we need to obtain the posterior predictive samples.

In [9]:
ppc1 = pm.sample_posterior_predictive(trace1, model=model1)
y_pred_ppc1 = az.extract(ppc1, group="posterior_predictive", var_names="y1_pred").values.T
az.r2_score(Y, y_pred_ppc1)

r2        0.871287
r2_std    0.014170
dtype: float64

In [10]:
ppc2 = pm.sample_posterior_predictive(trace2, model=model2)
y_pred_ppc2 = az.extract(ppc2, group="posterior_predictive", var_names="y2_pred").values.T
az.r2_score(Y, y_pred_ppc2)

r2        0.599941
r2_std    0.033100
dtype: float64

## Which model is better?
We can see that the r2 value of first model is greater than the second one, which means that **model 1 provides a better fit** to the data. Hence, the first model using **Petal Length** is better than the second one, because petal length has a greater impact in determining the petal width, comapred to sepal width.
